# Pairwise Reaction Energy Estimation #

For real protein complexes for which we have a crystal structure, we may want to estimate true reaction free energies in
order to get more realistic simulation results. This can be done using the Rosseta package, or specifically the pyrosetta
API.

This has three steps.

- The complex PDB is broken up into its subunits, each of which are saved as a separate pdb file.
- the individual subunits are relaxed using rosseta's `FastRelax` algorithm, which positions the residues in
the lowest energy conformations.
- all allowed pairwise reaction are preformed by joining two individual subunits into a new pdb structure, and
relaxing the bound structure again
- the reaction scores are computed as $S_{rxn} = S_{product} - S_{r1} - S_{r2}$
- These reaction scores are proportional to the $\Delta G_{rxn}$, although the scale might be drastically different

We only compute the pairwise scores because explicitly scoring the intermediate reaction using rosetta cause thermodynamic
inconsistencies due to instability and randomness in the relax algorithms. The energies for intermediate reactions are computed
from the pairwise reaction energies automatically when building a network using `ReactionNetwork.resolve_tree` from a `.pwr`
input file.

In [1]:
# make sure jupyter path is correct for loading local moudules
import sys
# path to steric_simulator module relative to notebook
sys.path.append("../../")
from steric_free_simulator import ReactionNetwork, EnergyExplorer

EnergyExplorer Module is not available. Check Rosetta installation. <ipykernel.iostream.OutStream object at 0x7ff7722b82b0>


ImportError: cannot import name 'EnergyExplorer' from 'steric_free_simulator' (../../steric_free_simulator/__init__.py)

First lets load the AP2 network from the input file

In [5]:
base_input = '../input_files/ap2.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()


Initialize an EnergyExplorer object and explore the network.

In [6]:
en = EnergyExplorer(rn, subunit_dir='../pdbs/AP2_subunits/')
en.explore_network()


PyRosetta-4 2020 [Rosetta PyRosetta4.Release.python37.ubuntu 2020.50+release.1295438cd4bd2be39c9dbbfab8db669ab62415ab 2020-12-12T00:30:01] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python37.ubuntu r274 2020.50+release.1295438cd4b 1295438cd4bd2be39c9dbbfab8db669ab62415ab http://www.pyrosetta.org 2020-12-12T00:30:01
core.init: command: PyRosetta -ex1 -ex2aro -database /home/spencerloggia/anaconda3/envs/asm_stoich/lib/python3.7/site-packages/pyrosetta-2020.50+release.1295438cd4b-py3.7-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-573203793 seed_offset=0 real_seed=-573203793
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-573203793 RG_type=mt19937
core.scoring.ScoreFunctionF

There will be a lot of output from rosseta, but the last lines should contain pairwise scores for each reaction.
These scores must be manually added as parameters in the `rules` section of the input file, scaled as needed. In
order to get a reaction network with these energies, you must reload the netwrok from the modified input file.
